In [19]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)

def nothing(x):
    pass
# Creating a window for later use
cv2.namedWindow('result')

# Starting with 100's to prevent error while masking
h_Max,h_Min,s_Max,s_Min,v_Max,v_Min = 179,0,255,0,255,0
# Creating track bar

cv2.createTrackbar('h_Max', 'result',0,179,nothing)
cv2.createTrackbar('h_Min', 'result',0,179,nothing)
cv2.createTrackbar('s_Max', 'result',0,255,nothing)
cv2.createTrackbar('s_Min', 'result',0,255,nothing)
cv2.createTrackbar('v_Max', 'result',0,255,nothing)
cv2.createTrackbar('v_Min', 'result',0,255,nothing)
cv2.createTrackbar('Threshold', 'result',0,1,nothing)
cv2.createTrackbar('Blur', 'result',0,1,nothing)
cv2.createTrackbar('Dilate', 'result',0,1,nothing)
cv2.createTrackbar('Open', 'result',0,1,nothing)
cv2.createTrackbar('Close', 'result',0,1,nothing)
cv2.createTrackbar('Erode', 'result',0,1,nothing)
cv2.createTrackbar('FindCountours', 'result',0,1,nothing)
cv2.createTrackbar('Area_Low', 'result',0,6000,nothing)
cv2.createTrackbar('Number', 'result',0,10,nothing)
#Neural Network Parameter : Training Accuracy:1 Testing Accuracy: 0.8
W = np.load('W.npy')
b = np.load('b.npy') 
dim = (28,28)
def getTrackbarPos(frame):
    #converting to HSV
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    h_Max = cv2.getTrackbarPos('h_Max','result')
    s_Max = cv2.getTrackbarPos('s_Max','result')
    v_Max = cv2.getTrackbarPos('v_Max','result')
    h_Min = cv2.getTrackbarPos('h_Min','result')
    s_Min = cv2.getTrackbarPos('s_Min','result')
    v_Min = cv2.getTrackbarPos('v_Min','result')
    return h_Max,h_Min,s_Max,s_Min,v_Max,v_Min
    # get info from track bar and appy to result
def bound(frame):
    h_Max,h_Min,s_Max,s_Min,v_Max,v_Min= getTrackbarPos(frame)
    lower_blue = np.array([h_Min,s_Min,v_Min])
    upper_blue = np.array([h_Max,s_Max,v_Max])
    return lower_blue,upper_blue
def GetMask(frame):
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    lower_blue,upper_blue = bound(frame)
    mask = cv2.inRange(hsv,lower_blue, upper_blue)
    return mask
def threshold(img):
    if cv2.getTrackbarPos('Threshold','result') == 1: #Switch
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        Tmask = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
        cv2.THRESH_BINARY,11,2)
        img = cv2.bitwise_and(img,img,mask = Tmask)
        return img
    else:
        return img
def Blur(img):
    if cv2.getTrackbarPos('Blur','result') == 1: #Switch
        img = cv2.medianBlur(img,5)
        return img
    else:
        return img
def Dilate(img):
    if cv2.getTrackbarPos('Dilate','result') == 1:#Switch
        kernel = np.ones((10,10),np.uint8)
        img = cv2.dilate(img,kernel,iterations = 1)
        return img
    else:
        return img
def Open(img):
    if cv2.getTrackbarPos('Open','result') == 1:#Switch
        kernel = np.ones((10,10),np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        return img
    else:
        return img
def Erode(img):
    if cv2.getTrackbarPos('Erode','result') == 1:#Switch
        kernel = np.ones((10,10),np.uint8)
        img = cv2.erode(img,kernel,iterations = 1)
        return img
    else:
        return img
def Close(img):
    if cv2.getTrackbarPos('Close','result') == 1:#Switch
        kernel = np.ones((10,10),np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
        return img
    else:
        return img
def Filter(frame):
    img = cv2.bitwise_and(frame,frame,mask = GetMask(frame)) #HSL Filtering
    img = Blur(img) #Blur image
    img = threshold(img) #Adaptive Threshold
    #Morphological Transformations
    img = Close(img)
    img = Dilate(img)
    img = Erode(img)
    img = Open(img) 
    return img
def countour(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _,contours,_ = cv2.findContours(gray,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    return contours
def subimage(image, centre, theta, width, height):
    output_image = cv2.CreateImage((width, height), image.depth, image.nChannels)
    mapping = np.array([[np.cos(theta), -np.sin(theta), centre[0]],
                       [np.sin(theta), np.cos(theta), centre[1]]])
    map_matrix_cv = cv2.fromarray(mapping)
    cv2.GetQuadrangleSubPix(image, output_image, map_matrix_cv)
    return output_image
def neuralprocess(W,b,Input,dim):
    Input = cv2.resize(Input,dim, interpolation = cv2.INTER_AREA)
    Input = Input.reshape((1,dim[0]*dim[1]))
    output = np.maximum((np.matmul(Input, W)+b),0)
    return np.argmax(output)
while(1):
    _, frame = cap.read()
    # Normal masking algorithm
    result = Filter(frame)
    #Label Contours
    if cv2.getTrackbarPos('FindCountours','result')==1:
        for cnt in countour(result): 
            x,y,w,h = cv2.boundingRect(cnt)
            gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
            if cv2.contourArea(cnt)>=cv2.getTrackbarPos('Area_Low','result'):
                if neuralprocess(W,b,gray[y:y+h,x:x+w],dim)==cv2.getTrackbarPos('Number','result'): #Represents NUMBER 0
                    cv2.rectangle(result,(x,y),(x+w,y+h),(255,255,255),2)
                else:
                    cv2.rectangle(result,(x,y),(x+w,y+h),(0,0,255),2)
            cv2.imshow('Crop',gray[y:y+h,x:x+w])
    cv2.imshow('result',result)
    cv2.imshow('Result',result)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

cap.release()

cv2.destroyAllWindows()


3
3
3
3
3
3
2
3
5
5
2
3
3
3
3
3
5
5
3
3
3
3
3
3
3
3
3
3
3
3
5
5
3
3
5
3
5
3
3
3
3
3
5
3
3
3
5
3
3
3
5
2
2
5
3
3
3
3
3
3
5
2
5
3
3
3
3
3
3
3
3
3
3
3
3
3
5
3
3
3
3
3
3
3
5
3
3
3
2
5
5
3
3
3
3
3
3
3
2
3
3
3
3
3
3
3
3
3
3
3
3
2
2
5
3
2
3
3
3
5
3
3
3
3
3
3
2
3
3
3
3
3
3
3
3
3
3
6
3
3
5
3
3
3
3
3
3
3
3
5
3
5
2
3
3
3
5
8
3
3
3
3
3
5
3
2
5
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
3
3
3
3
3
3
3
5
3
3
3
3
2
3
3
3
5
3
5
3
3
3
3
3
3
2
5
2
3
3
3
3
3
3
3
3
3
3
5
3
2
3
3
3
3
3
3
3
3
5
3
2
5
3
3
3
3
3
3
3
3
3
3
5
3
3
3
3
5
2
3
3
5
3
2
3
3
3
3
3
3
5
3
8
3
3
5
3
3
3
2
3
3
3
8
3
3
3
3
3
3
5
5
3
3
3
5
5
2
3
3
3
5
3
3
3
3
5
3
3
5
3
3
5
3
2
5
3
3
3
3
5
3
5
3
3
3
3
5
5
3
5
3
5
2
5
3
3
3
2
3
5
3
2
3
3
3
3
5
5
3
3
3
5
3
2
5
3
3
3
3
2
3
3
3
5
3
3
3
3
3
5
3
3
3
3
2
6
3
3
5
2
3
3
3
3
3
5
3
3
3
3
3
2
3
5
5
5
3
3
3
3
3
2
3
2
8
3
3
3
3
3
3
3
3
3
3
3
3
5
5
3
3
3
3
3
3
8
3
3
5
3
8
3
3
3
2
3
3
2
3
3
6
3
5
3
3
3
2
3
3
3
2
3
3
5
0
3
3
3
3
2
6
3
5
3
2
3
2
3
3
3
3
3
3
3
5
3
2
2
5
3
2
3
3
3
3
3
3
3
3
3
3
3
3
3
2
3
2
5
3
2
3
3
3
5
5
3


KeyboardInterrupt: 

In [ ]:
#High error prob.

In [1]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

For HSV, Hue range is [0,179], Saturation range is [0,255] and Value range is [0,255]
More Labeling : http://docs.opencv.org/3.1.0/dd/d49/tutorial_py_contour_features.html

Parameters for Face
19
0
214
68
148
0
0
1
1
0
1